<a href="https://colab.research.google.com/github/RoAlro/IA-codes-projects/blob/main/Codigo_para_la_camara_y_GUI_del_Clasificador_Basura_Reciclable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **ALGORITMO DE CLASIFICACIÓN DE BASURA RECICLABLE**

In [ ]:
import numpy as np
from tkinter import *
import cv2
import serial
import imutils as im
from PIL import Image
from PIL import ImageTk
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import timm
import time

In [ ]:
class CNN_RESNET34(nn.Module):
    def __init__(self, num_classes = 4, pretrained = True):
        super().__init__()
        
        self.num_classes = num_classes

        # Para MOBILENET
        self.resnet34 = timm.create_model("resnet34d", pretrained = pretrained)
        if pretrained:
          # freeze  weights
            for param in self.resnet34.parameters():
              param.requires_grad = False 

        self.numfeat = self.resnet34.get_classifier().in_features
        
        block = nn.Sequential(
            nn.Linear(self.numfeat, 1024), 
            nn.Dropout(0.3),
            nn.Linear(1024, self.num_classes))                 

        self.resnet34.fc = block       

    def forward(self, x):
        out = self.resnet34.forward(x)
        return out

In [ ]:
def predecir_clase(imagen):
    # Crear una instancia del modelo
    model = CNN_RESNET34() 

    dir_model = "C:/Users/UsuarioE/Desktop/PROYECTO IA/Modelo/NetModel3.pth"

    # Cargar los pesos entrenados entrenado
    state_dict = torch.load(dir_model, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)

    # Configura el modelo en modo de evaluación
    model.eval()

    # Aplica las transformaciones necesarias
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Preparar la imagen de entrada
    img_tensor = transform(imagen)
    img_tensor = img_tensor.unsqueeze(0) # add batch dimension

    # Hacer la predicción
    with torch.no_grad():
        output = model(img_tensor)  # Pasar la imagen a traves del modelo
        _, pred = torch.max(output, 1)  # Obtiene la clase con la probabilidad más alta
        score = output.numpy()[0,pred.item()]

    return  pred.item(), score

In [ ]:
# Inicializacion de variables
captura = None
servos_giro_radial = 0
servos_inclinación = 0
angulo_giro_radial = 135
angulo_inclinación = 0
generar_clasificación = False
posicion_inicial = False

In [ ]:
# Establece la conexión serie en el puerto COM3 a 9600 baudios
arduino = serial.Serial('COM3', 9600, timeout = 1)

In [ ]:
def girar_Servos(servos_giro_radial_1, servos_inclinación_1, angulo_giro_radial_1, angulo_inclinación_1):
    global arduino
    datos = str(servos_giro_radial_1)+","+str(servos_inclinación_1)+","+str(angulo_giro_radial_1)+","+str(angulo_inclinación_1)
    arduino.write(datos.encode())

In [ ]:
def boton_posicion_inicial():
    global angulo_giro_radial, angulo_inclinación, posicion_inicial

    posicion_inicial = True
    angulo_giro_radial = 135
    angulo_inclinación = 0
    girar_Servos(1, 1, angulo_giro_radial, angulo_inclinación)
    time.sleep(2)

In [ ]:
def movimiento_clasificador():
    global angulo_giro_radial, angulo_inclinación

    girar_Servos(1, 0, angulo_giro_radial, 0)
    print(angulo_giro_radial)
    time.sleep(2)

    girar_Servos(0, 1, angulo_giro_radial, 30)
    print('ang incl 30')
    time.sleep(2.5)

    girar_Servos(0, 1, angulo_giro_radial, 0)
    print('ang incl 0')
    time.sleep(1.5)

    girar_Servos(1, 0, 135, 0)
    print(135)
    time.sleep(1.5)

In [ ]:
def clasificar(imagen):
    global lblEtiquetaClase, angulo_giro_radial, angulo_inclinación
    # Predicir la clase (numero) de la imagen de video, ademas de obtener su score o puntaje
    num_clase, score = predecir_clase(imagen)
    
    # Verificar si es uno de las 4 clase de basura reciclable 
    if score >= 3:
        if num_clase == 0:
            lblEtiquetaClase.configure(text = 'Botella de PET')
            angulo_giro_radial = 260
            movimiento_clasificador()

        elif num_clase == 1:
            lblEtiquetaClase.configure(text = 'Botella de Vidrio')
            angulo_giro_radial = 198
            movimiento_clasificador()

        elif num_clase == 2:
            lblEtiquetaClase.configure(text = 'Vaso o Frasco de Vidrio')
            angulo_giro_radial = 73
            movimiento_clasificador()

        #elif num_clase == 3:
            #lblEtiquetaClase.configure(text = 'Vaso o Frasco de Vidrio')
            #angulo_giro_radial = 10
            #movimiento_clasificador()

    else:
        lblEtiquetaClase.configure(text = 'Ninguna de las 3')
        angulo_giro_radial = 135
        movimiento_clasificador()

In [ ]:
def boton_clasificador():
    global generar_clasificación
    generar_clasificación = True
    print('Clasificando')

In [ ]:
def video():
    global lblVideo, captura, generar_clasificación, posicion_inicial

    if captura is not None:
        # Leer imagen de la cámara
        exito, imagen = captura.read(0)

        if not exito:
            print('No se ha podido tomar la captura')
            lblVideo.image = ""
        else:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB) 

            # Girar imagen desde el eje vertical
            imagen = cv2.flip(imagen, 1)
            
            if generar_clasificación == True and posicion_inicial == True:
                # Clasificar imagen de basura 
                clasificar(imagen)
                generar_clasificación = False
                print('Clasificado')

            # Convertimos la imagen
            img1 = Image.fromarray(imagen)
            img2 = ImageTk.PhotoImage(image=img1)

            # Mostramos la imagen procesada en la GUI
            lblVideo.configure(image=img2)
            lblVideo.image = img2
            lblVideo.after(10, video)

In [ ]:
# Crear la ventana de la GUI
root = Tk()
root.title('CLASIFICADOR DE BASURA RECICLABLE')
root.geometry('910x580')
root.resizable(0, 0)

# Creando Widgets
lblTitulo = Label(root, text = 'CLASIFICADOR DE BASURA RECICLABLE', fg = 'red', font = ('Cambria',24))
lblTitulo.place(x = 170, y = 10)

btnVideo = Button(root, text="ACTIVAR CAMARA", font = ('Cambria',16), command=video)
btnVideo.place(x = 20, y = 90, width=200, height=35)

btnPosicionIncial = Button(root, text="POSICIÓN INICIAL", font = ('Cambria',16), command=boton_posicion_inicial)
btnPosicionIncial.place(x = 20, y = 140, width=200, height=35)

btnClasificador = Button(root, text="CLASIFICAR", font = ('Cambria',16), command=boton_clasificador)
btnClasificador.place(x = 20, y = 190, width=200, height=35)

lblVideo = Label(root)
lblVideo.place(x = 240, y = 70)

lblClase = Label(root, text = 'CLASE:', fg = 'green', font = ('Cambria',16))
lblClase.place(x = 20, y = 260)

lblEtiquetaClase = Label(root, text = '', fg = 'blue', font = ('Cambria',16))
lblEtiquetaClase.place(x = 20, y = 300)

# Capturar video
captura = cv2.VideoCapture(1)

# Bucle infinito siempre cuando no se cierre la ventana de la GUI
root.mainloop()

captura.release()
arduino.close()

135
ang incl 30
ang incl 0
135
Clasificado
Clasificando
260
ang incl 30
ang incl 0
135
Clasificado
Clasificando
73
ang incl 30
ang incl 0
135
Clasificado
Clasificando
198
ang incl 30
ang incl 0
135
Clasificado
Clasificando
135
ang incl 30
ang incl 0
135
Clasificado
Clasificando
135
ang incl 30
ang incl 0
135
Clasificado
Clasificando
260
ang incl 30
ang incl 0
135
Clasificado
Clasificando
135
ang incl 30
ang incl 0
135
Clasificado
Clasificando
198
ang incl 30
ang incl 0
135
Clasificado
Clasificando
260
ang incl 30
ang incl 0
135
Clasificado
Clasificando
198
ang incl 30
ang incl 0
135
Clasificado
Clasificando
135
ang incl 30
ang incl 0
135
Clasificado
Clasificando
73
ang incl 30
ang incl 0
135
Clasificado
Clasificando
135
ang incl 30
ang incl 0
135
Clasificado
Clasificando
73
ang incl 30
ang incl 0
135
Clasificado
Clasificando
198
ang incl 30
ang incl 0
135
Clasificado
Clasificando
135
ang incl 30
ang incl 0
135
Clasificado
Clasificando
260
ang incl 30
ang incl 0
135
Clasificado
Clasific